In [1]:
import pandas as pd

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import torch
import numpy as np
import pickle as pk

from tqdm import tqdm_notebook
from sklearn.metrics import cohen_kappa_score
from fastai.vision import *
from torch.nn import functional as F
from utils import *

In [2]:
save_dir = "../output/blending"

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = 2019
seed_everything(SEED)

deployment_dir = "../output/inference"

def qk(y_pred, y):
    k = torch.tensor(cohen_kappa_score(torch.round(y_pred), y, weights='quadratic'), device='cuda:0')
    k[k != k] = 0
    k[torch.isinf(k)] = 0
    
    return k


In [4]:
test_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [6]:
models = ["efficientnet-b4_0820_01-09-57_stage2_f4", "efficientnet-b4_0823_18-14-19_stage2_whole", 
             "efficientnet-b5_0823_18-21-16_stage2_whole", "efficientnet-b5_0821_01-30-37_stage2_f4",
            "efficientnet-b5_0820_22-13-07_stage2_f3"]

for m in models:
    learn = load_learner(deployment_dir, "{}.pkl".format(m))

    learn.data.add_test(ImageList.from_df(test_df,
                                      '../input/aptos2019-blindness-detection',
                                      folder='test_images_ben_preprocessing_sigmaX10',
                                      suffix='.png'))

    logits,_ = learn.get_preds(DatasetType.Test)
    logits = logits.numpy()
    np.save(os.path.join(save_dir, m+".npy"), logits)
    y_pred = np.round(logits)
    test_df.diagnosis = y_pred
    test_df.to_csv(os.path.join(save_dir, m+".csv"), index=False)